In [3]:
!nvidia-smi

Thu Oct  5 14:32:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Fine-tuning Mistral 7b on Medical Data



### Setup Runtime
For fine-tuning Llama, a GPU instance is essential. Follow the directions below:

1. Go to `Runtime` (located in the top menu bar).
2. Select `Change Runtime Type`.
3. Choose `T4 GPU` (or a comparable option).


### Package Installation


In [4]:
!pip install -q pandas


In [5]:
!pip install -q autotrain-advanced


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 44.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 59.0 MB/s eta 0

### Setup Autotrain
The step below is required for AutoTrain in Colab


In [6]:
!autotrain setup --update-torch


> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


### Connect to Hugging Face for model upload
#### Logging to Hugging Face


##### Getting a Hugging Face token
Steps:
1. Navigate to this URL: https://huggingface.co/settings/tokens
2. Create a `write` token and copy it to your clipboard
3. Run the code below and enter your token



In [7]:
from huggingface_hub import notebook_login
notebook_login()

### Upload your dataset
Add your data set to the root directory in the Colab under the name `train.csv`. The AutoTrain command will look for your data there under that name.

##### Don't have a data set and want to try finetuning on an example data set?
If you don't have a dataset you can run these commands below to get an example data set and save it to `train.csv`






In [8]:
from datasets import load_dataset

#Loading Medical Dataset
dataset = load_dataset("BI55/MedText")

print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Prompt', 'Completion'],
        num_rows: 1412
    })
})


In [9]:
import pandas as pd
medical_data=pd.DataFrame(dataset['train'])
medical_data.head()

,Prompt,Completion
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan..."
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...


In [10]:
# Converting data to model aligned fine-tuning

medical_data['text']="###Human:\n"+medical_data['Prompt']+"\n\n"+"###Assistant:\n"+medical_data['Completion']
medical_data

,Prompt,Completion,text
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...,###Human:\nA 50-year-old male presents with a ...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan...",###Human:\nA 7-year-old boy presents with a fe...
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...,###Human:\nA 35-year-old woman presents with a...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...,###Human:\nA 50-year-old male presents with se...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...,###Human:\nA newborn baby presents with eye re...
...,...,...,...
1407,A 55-year-old male with a history of chronic o...,While this patient's symptoms could be due to ...,###Human:\nA 55-year-old male with a history o...
1408,Can diet and lifestyle changes help manage vit...,While there is no specific diet or lifestyle m...,###Human:\nCan diet and lifestyle changes help...
1409,A 50-year-old female presents with right shoul...,This patient's shoulder and arm pain following...,###Human:\nA 50-year-old female presents with ...
1410,A 60-year-old female with high cholesterol lev...,In addition to a diet low in saturated fats an...,###Human:\nA 60-year-old female with high chol...


In [11]:
medical_data['text'][0]

"###Human:\nA 50-year-old male presents with a history of recurrent kidney stones and osteopenia. He has been taking high-dose vitamin D supplements due to a previous diagnosis of vitamin D deficiency. Laboratory results reveal hypercalcemia and hypercalciuria. What is the likely diagnosis, and what is the treatment?\n\n###Assistant:\nThis patient's history of recurrent kidney stones, osteopenia, and high-dose vitamin D supplementation, along with laboratory findings of hypercalcemia and hypercalciuria, suggest the possibility of vitamin D toxicity. Excessive intake of vitamin D can cause increased absorption of calcium from the gut, leading to hypercalcemia and hypercalciuria, which can result in kidney stones and bone loss. Treatment would involve stopping the vitamin D supplementation and potentially providing intravenous fluids and loop diuretics to promote the excretion of calcium."

In [12]:
medical_data.to_csv('train.csv',index=False)

## Overview of AutoTrain command

#### Short overview of what the command flags do.

- `!autotrain`: Command executed in environments like a Jupyter notebook to run shell commands directly. `autotrain` is an automatic training utility.

- `llm`: A sub-command or argument specifying the type of task

- `--train`: Initiates the training process.

- `--project_name`: Sets the name of the project

- `--model abhishek/llama-2-7b-hf-small-shards`: Specifies original model that is hosted on Hugging Face named "llama-2-7b-hf-small-shards" under the "abhishek".

- `--data_path .`: The path to the dataset for training. The "." refers to the current directory. The `train.csv` file needs to be located in this directory.

- `--use_int4`: Use of INT4 quantization to reduce model size and speed up inference times at the cost of some precision.

- `--learning_rate 2e-4`: Sets the learning rate for training to 0.0002.

- `--train_batch_size 12`: Sets the batch size for training to 12.

- `--num_train_epochs 3`: The training process will iterate over the dataset 3 times.

### Steps needed before running
Go to the `!autotrain` code cell below and update it by following the steps below:

1. After `--project_name` replace `*enter-a-project-name*` with the name that you'd like to call the project
2. After `--repo_id` replace `*username*/*repository*`. Replace `*username*` with your Hugging Face username and `*repository*` with the repository name you'd like it to be created under. You don't need to create this repository before hand, it will automatically be created and uploaded once the training is completed.
3. Confirm that `train.csv` is in the root directory in the Colab. The `--data_path .` flag will make it so that AutoTrain looks for your data there.
4. Make sure to add the LoRA Target Modules to be trained `--target-modules q_proj, v_proj`
5. Once you've made these changes you're all set, run the command below!




In [ ]:
!autotrain llm --train --project_name mistral-7b-medicalgpt --model bn22/Mistral-7B-Instruct-v0.1-sharded --data_path . --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 4 --num_train_epochs 3 --trainer sft --target_modules q_proj,v_proj --push_to_hub --repo_id Naveen5/mistral-7b-medigpt-finetuned

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='.', train_split='train', valid_split=None, text_column='text', model='bn22/Mistral-7B-Instruct-v0.1-sharded', learning_rate=0.0002, num_train_epochs=3, train_batch_size=4, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='mistral-7b-medicalgpt', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=True, use_int8=False, model_max_length=1024, repo_id='Naveen5/mistral-7b-medigpt-finetuned', use_int4=True, trainer='sft', target_modules='q_proj,v_proj', merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=False, func=<function run_llm_command_factory at 

# Inference


In [ ]:

!autotrain llm -h

In [ ]:
!pip install -q safetensors

In [ ]:
pip install accelerate

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistral-7b-medigpt-finetuned"
#adapters_name = "Naveen5/mistral-7b-medigpt-finetuned"

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,

    torch_dtype=torch.bfloat16,
    device_map='auto',
    offload_folder = "folder/")
#model = PeftModel.from_pretrained(model, adapters_name)
#model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

In [ ]:

text = "I am suffering fever from past few weeks with body pains and vomitings"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# model_inputs = encodeds.to(device)
model_inputs = encodeds
# model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])